# Analyzing planning trials

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 100 scenes, (30 grasps each scene (we are not following this strictly, check the paper for details)), 10 runs

def get_method_exp_paths(base_path):
    methods = []
    for method_path in os.listdir(base_path):
        if 'dbg' in method_path:
            continue

        print(method_path)
        method = {
            'name': method_path,
            'exps': []
        }
        exp_paths = os.listdir(f'{base_path}/{method_path}')

        # Check that all experiments have 10 runs
        if len(exp_paths) != 10:
            print(f'Warning: {len(exp_paths)} total runs for {method_path}')

        for exp_path in exp_paths:
            scene_paths = [x for x in os.listdir(f'{base_path}/{method_path}/{exp_path}') if 'args.yml' not in x]
            print(f"{method_path}/{exp_path} has {len(scene_paths)} scenes")
#             if len(scene_paths) < 100:
#                 print(f"Warning: skipping {method_path}/{exp_path} as there are only {len(scene_paths)} scenes")
#                 continue
            method['exps'].append(exp_path)
        methods.append(method)
    return methods
            
def get_method_results(base_path):
    """
    Returns evaluation metrics for a grasp inference method 
    Run on 10 runs of 100 scenes. 
    Evaluation metrics: execution success, planning success, smoothness, collision, time
    Returns a pandas dataframe with each eval metric in its own column
    """
    methods = get_method_exp_paths(base_path)
    runs_metrics = {
        'method': [],
        'exec': [],
        'plan': [],
        'smth': [],
        'coll': [],
        'plan_time': [],
#         'infer_time': []
    }
#   TODO load data.npy, collect metrics, return data frame
    for method in methods:
        for exp_path in method['exps']:
            path = f'{base_path}/{method["name"]}/{exp_path}'
            scene_paths = os.listdir(path)
            
            exp_metrics = {
                'method': method["name"],
                'exec': [],
                'plan': [],
                'smth': [],
                'coll': [],
                'plan_time': [],
#                 'infer_time': []
            }
            for i, scene_path in enumerate(scene_paths):
                if 'args.yml' in scene_path:
                    continue

                try:
                    data = np.load(f'{path}/{scene_path}/data.npy', allow_pickle=True, encoding='latin1')
                except Exception as e:
                    print(e)
                    print(f"Stopping early at {i}")
                    break
#                 rew, info, plan, infer_time = data
                rew, info, plan = data
                if len(info) < 1:
                    continue
                
                exp_metrics['exec'].append(rew)
                exp_metrics['plan'].append(1 if info[-1]['terminate'] else 0)
                exp_metrics['smth'].append(info[-1]['smooth'])
                exp_metrics['coll'].append(info[-1]['obs'])
                exp_metrics['plan_time'].append(info[-1]['time'])
#                 exp_metrics['infer_time'].append(infer_time)
            runs_metrics['method'].append(exp_metrics['method'])
            runs_metrics['exec'].append(np.mean(exp_metrics['exec']))
            runs_metrics['plan'].append(np.mean(exp_metrics['plan']))
            runs_metrics['smth'].append(np.mean(exp_metrics['smth']))
            runs_metrics['coll'].append(np.mean(exp_metrics['coll']))
            runs_metrics['plan_time'].append(np.mean(exp_metrics['plan_time']))
#             runs_metrics['infer_time'].append(np.mean(exp_metrics['infer_time']))
#         break
#     print(runs_metrics)
    m_df = pd.DataFrame(runs_metrics)
    print(m_df.round(3))
    
    return m_df
            
runpaths = '/data/manifolds/pybullet_eval/saved_results/goalsetproj_vs_softcost'
df = get_method_results(runpaths)

_bak
_bak/implicitgrasps_goalgrad_fixedgoal has 1 scenes
_bak/implicitgrasps_goalsetproj has 1 scenes
_bak/implicitgrasps_goalsetproj_fixedgoal has 1 scenes
knowngrasps_grad_goalsetproj
knowngrasps_grad_goalsetproj/2022-02-17-01-19-45_implicitgrasps_outputposegradfixed has 5 scenes
predgrasps_grad_goalgrad
predgrasps_grad_goalgrad/2022-02-17-00-38-45_implicitgrasps_outputposegrad has 5 scenes
[Errno 2] No such file or directory: '/data/manifolds/pybullet_eval/saved_results/goalsetproj_vs_softcost/_bak/implicitgrasps_goalgrad_fixedgoal/2022-02-17-00-58-26_implicitgrasps_outputposegradfixed/data.npy'
Stopping early at 0
[Errno 2] No such file or directory: '/data/manifolds/pybullet_eval/saved_results/goalsetproj_vs_softcost/_bak/implicitgrasps_goalsetproj/2022-02-16-13-39-17_implicitgrasps_outputposegradfixed/data.npy'
Stopping early at 0
[Errno 2] No such file or directory: '/data/manifolds/pybullet_eval/saved_results/goalsetproj_vs_softcost/_bak/implicitgrasps_goalsetproj_fixedgoal/202

/home/exx/miniconda3/envs/gm_pipeline/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/exx/miniconda3/envs/gm_pipeline/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [3]:
def get_mean_std_str(df):
    means = list(df[['exec', 'plan', 'smth', 'coll', 'plan_time', 'infer_time']].mean().values.round(3))
    stds = list(df[['exec', 'plan', 'smth', 'coll', 'plan_time', 'infer_time']].std().values.round(3))
    return [f"{m}+{s}" for m, s in zip(means, stds)]

df = pd.DataFrame(columns=acronym_df.columns)
df.loc[0] = ['acronym_Fixed'] + get_mean_std_str(acronym_df.loc[acronym_df['method'] == "acronym_Fixed"])
df.loc[1] = ['acronym_Proj'] + get_mean_std_str(acronym_df.loc[acronym_df['method'] == "acronym_Proj"])
df.loc[2] = ['acronym_OMG'] + get_mean_std_str(acronym_df.loc[acronym_df['method'] == "acronym_OMG"])
df.loc[3] = ['contact_graspnet_Fixed'] + get_mean_std_str(contactgraspnet_df.loc[contactgraspnet_df['method'] == "contact_graspnet_Fixed"])
df.loc[4] = ['contact_graspnet_Proj'] + get_mean_std_str(contactgraspnet_df.loc[contactgraspnet_df['method'] == "contact_graspnet_Proj"])
df.loc[5] = ['contact_graspnet_OMG'] + get_mean_std_str(contactgraspnet_df.loc[contactgraspnet_df['method'] == "contact_graspnet_OMG"])

df

,method,exec,plan,smth,coll,plan_time,infer_time
0,acronym_Fixed,0.931+0.016,0.977+0.013,12.833+0.539,11.648+1.785,0.046+0.012,0.0+0.0
1,acronym_Proj,0.914+0.023,0.98+0.013,7.957+0.134,9.785+1.077,0.042+0.009,0.0+0.0
2,acronym_OMG,0.926+0.025,0.97+0.017,13.354+0.384,12.639+2.033,0.176+0.028,0.0+0.0
3,contact_graspnet_Fixed,0.31+0.035,0.979+0.01,13.639+0.586,4.824+0.748,0.042+0.008,0.325+0.012
4,contact_graspnet_Proj,0.333+0.042,0.988+0.01,8.477+0.173,6.06+1.375,0.037+0.007,0.327+0.011
5,contact_graspnet_OMG,0.341+0.047,0.983+0.014,13.469+0.761,4.389+1.358,0.114+0.022,0.325+0.001
